<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/data/data-augmentor/KoGPT-2%20Data%20Augmentor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# Load packages
import io
import csv
import re
import random
import torch
import numpy as np
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, AutoTokenizer

# Set Data to be Augmented

In [3]:
from google.colab import files

uploaded = files.upload()

Saving training.csv to training.csv


업로드 된 자료 중에서 긍정적인 감정이 있는 행(`label==1`)만을 이용해서 Data Augmentation을 진행하겠습니다.

In [6]:
# Get the content of the uploaded file
file_content = next(iter(uploaded.values()))

# Convert bytes to string
file_content_str = file_content.decode('utf-8')

# Iterate through each line in the file
pos_label_rows = []

for line in file_content_str.split('\n'):
    columns = line.split('\t')

    # Check if the label is 1
    if len(columns) > 0 and columns[0] == '1':
        pos_label_rows.append(line)

추출된 데이터 중 처음 10개의 예시를 확인해 보겠습니다.

In [8]:
pos_label_rows[:10]

['1\t퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.\r',
 '1\t요즘 직장생활이 너무 편하고 좋은 것 같아!\r',
 '1\t취업해야 할 나이인데 취업하고 싶지가 않아.\r',
 '1\t우리 회사는 정말 사내 분위기가 좋아.\r',
 '1\t오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.\r',
 '1\t회사에서 전공시험을 봤어. 오늘 시험 결과가 나왔어.\r',
 '1\t오늘 입사 면접을 봤어.\r',
 '1\t회사에서 나를 참 신뢰하는 것 같아. 그건 기분이 좋아.\r',
 '1\t먼저 취업한 선배가 면접 비결을 알려줬어. 곧 면접 보러 가는데 든든해.\r',
 '1\t직장 상사로부터 칭찬을 받았는데 너무 신이 나!\r']

최종적으로 총 6126개의 긍정적인 태그가 지정된 예시가 있습니다.

In [9]:
print(len(pos_label_rows))

6126


# Augment sentences

다음으로 KoGPT-2의 Language Model을 이용해서 긍정적인 문장을 생성합니다. 아래의 두 가지 가정으로부터 생성된 문장들이 긍정적인 감정을 나타낸다고 판단하였습니다.

1. 텍스트 문장에 포함된 각 단어는 긍정적인지 부정적인지의 감정을 전달한다.
2. 긍정적인 감정을 나타내는 단어들은 긍정적인 감정의 다른 단어들과 함께 나타날 가능성이 높다.

생성하는 시퀀스의 최대 길이는 50으로 하였고, Seed Sentence를 입력 하였을 때, 매 시점마다 Top 5개의 단어들(`top_k=5`) 중에서 랜덤으로 선택하는 방식을 택하여 문장을 생성하였습니다.

In [57]:
def generate_sentence_with_top_k(seed_sentence, model, tokenizer, max_length=50, top_k=5):
    # Remove '1\t' prefix
    seed_sentence = seed_sentence.lstrip('1\t')

    input_ids = tokenizer.encode(seed_sentence, return_tensors='tf')

    for _ in range(max_length - len(input_ids[0])):
        output = model(input_ids)

        # Get the top k indices and randomly select the next token from the selected indices
        top_k_indices = tf.math.top_k(output.logits[0, -1], k=top_k).indices.numpy()
        selected_token_id = np.random.choice(top_k_indices)
        input_ids = tf.concat([input_ids, tf.constant([[selected_token_id]])], axis=-1)

    generated_sentence = tokenizer.decode(input_ids[0, len(tokenizer.encode(seed_sentence, return_tensors='tf')):])

    # Remove the seed sentence from the generated text using .replace
    generated_sentence = generated_sentence.replace(seed_sentence, '')

    # Replace newline characters by space
    generated_sentence = generated_sentence.replace('\n', '\s')

    # Remove non-Korean characters using re.sub
    generated_sentence = re.sub(r'[^가-힣\s!?.]', '', generated_sentence)

    return generated_sentence.strip()

def main(sentences, output_filename):
    model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

    # Use AutoTokenizer to load the tokenizer associated with the model
    tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

    generated_sentences = []

    for seed_sentence in sentences:
        generated_sentence = generate_sentence_with_top_k(seed_sentence, model, tokenizer)
        generated_sentences.append({
            'seed_sentence': seed_sentence,
            'generated_sentence': generated_sentence
        })
        print(f"Seed Sentence: {seed_sentence}\nGenerated Sentence: {generated_sentence}\n")

    with open(output_filename, 'w', encoding='utf-8', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter='\t')
        csv_writer.writerow(['emotion', 'sentence'])

        for data in generated_sentences:
            csv_writer.writerow(['1', data['generated_sentence']])

In [59]:
# Example usage:
data_list = ['1\t퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.\r',
             '1\t요즘 직장생활이 너무 편하고 좋은 것 같아!\r',
             '1\t취업해야 할 나이인데 취업하고 싶지가 않아.\r',
             '1\t우리 회사는 정말 사내 분위기가 좋아.\r',
             '1\t오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.\r',
             '1\t회사에서 전공시험을 봤어. 오늘 시험 결과가 나왔어.\r',
             '1\t오늘 입사 면접을 봤어.\r',
             '1\t회사에서 나를 참 신뢰하는 것 같아. 그건 기분이 좋아.\r',
             '1\t먼저 취업한 선배가 면접 비결을 알려줬어. 곧 면접 보러 가는데 든든해.\r',
             '1\t직장 상사로부터 칭찬을 받았는데 너무 신이 나!\r']

generated_data = main(data_list, 'test.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.
Generated Sentence: 사한 지 얼마 안 됐지만 천천히 직장을 구해보려고. 오늘부터 다시 시작아이스커피머핀카카오에브리데이오늘

Seed Sentence: 1	요즘 직장생활이 너무 편하고 좋은 것 같아!
Generated Sentence: 직장생활이 너무 편하고 좋은 것 같아!.이번에 신랑이 사온 라면  라면은 정말 맛있어!..오늘은 신랑

Seed Sentence: 1	취업해야 할 나이인데 취업하고 싶지가 않아.
Generated Sentence: 해야 할 나이인데 취업하고 싶지가 않아.. 이럴때마다 내가 제일 걱정되는 것이 뭔데? 나랑 같은 나이인데. 나만 잘난 것 같고. 이게 왠만해서 저는

Seed Sentence: 1	우리 회사는 정말 사내 분위기가 좋아.
Generated Sentence: 회사는 정말 사내 분위기가 좋아.. 뭔들 그렇게 좋아할까?하고 말하면서 냅두고는 괜찮은 곳으로 갔다.그리고 그 곳에 가서 뭘 하고 있는지 살펴보려고 다시 한 시간 동안 걸음을 멈추었다.그때까지

Seed Sentence: 1	오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.
Generated Sentence: 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.그렇게 열심히 일하는 동안 나는 늘 행복했었다면서 .오늘은이런 이런날  요즘 너무 귀욤뽀

Seed Sentence: 1	회사에서 전공시험을 봤어. 오늘 시험 결과가 나왔어.
Generated Sentence: 전공시험을 봤어. 오늘 시험 결과가 나왔어.?그때 내가 그 얘기를 했습니다.그런 얘기 하지 말랬어. 그건 내 전공 시험이 아니야. 나는 그걸 알고 있어

Seed Sentence: 1	오늘 입사 면접을 봤어.
Generated Sentence: 입사 면접을 봤어..그랬어.응? 뭔 소리냐? 뭔 소리야. 뭔 소리? 응응? 나는 고개를 저었다.응! 그 가 뭘 어쨌는지

Seed Sentence: 1	

In [61]:
import pandas as pd

test_dat = pd.read_csv('test.csv', sep='\t')
test_dat.head(10)

,emotion,sentence
0,1,사한 지 얼마 안 됐지만 천천히 직장을 구해보려고. 오늘부터 다시 시작아이스커피머핀...
1,1,직장생활이 너무 편하고 좋은 것 같아!.이번에 신랑이 사온 라면 라면은 정말 맛있...
2,1,해야 할 나이인데 취업하고 싶지가 않아.. 이럴때마다 내가 제일 걱정되는 것이 뭔데...
3,1,회사는 정말 사내 분위기가 좋아.. 뭔들 그렇게 좋아할까?하고 말하면서 냅두고는 괜...
4,1,내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.그렇게 열심히 일하는 동안 나...
5,1,전공시험을 봤어. 오늘 시험 결과가 나왔어.?그때 내가 그 얘기를 했습니다.그런 얘...
6,1,입사 면접을 봤어..그랬어.응? 뭔 소리냐? 뭔 소리야. 뭔 소리? 응응? 나는 고...
7,1,나를 참 신뢰하는 것 같아. 그건 기분이 좋아..그렇게 해서 내가 원하는 건 내가 ...
8,1,취업한 선배가 면접 비결을 알려줬어. 곧 면접 보러 가는데 든든해.그래서 저희도 햄...
9,1,상사로부터 칭찬을 받았는데 너무 신이 나!그럼에도 불구하고 이번에 받은 립스틱 하나...


Google colab에서 disconnected 되는 경우에 대비해서 전체 파일을 10등분 해서 각각 약 600여개의 문장으로 새로운 문장을 생성하겠습니다.

In [62]:
chunk1 = pos_label_rows[:601]
chunk2 = pos_label_rows[601:1201]
chunk3 = pos_label_rows[1201:1801]
chunk4 = pos_label_rows[1801:2401]
chunk5 = pos_label_rows[2401:3001]
chunk6 = pos_label_rows[3001:3601]
chunk7 = pos_label_rows[3601:4201]
chunk8 = pos_label_rows[4201:4801]
chunk9 = pos_label_rows[4801:5401]
chunk10 = pos_label_rows[5401:]

In [63]:
# Apply the main function to each chunk
main(chunk1, 'augmented_data_chunk1.csv')
files.download('augmented_data_chunk1.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.
Generated Sentence: 사한 지 얼마 안 됐지만 천천히 직장을 구해보려고..그냥 이게 다 좆같은데.. 이럴 좆같이 생겼는데.??? 가 좆 같은게 네한테 그런걸

Seed Sentence: 1	요즘 직장생활이 너무 편하고 좋은 것 같아!
Generated Sentence: 직장생활이 너무 편하고 좋은 것 같아! 다히로 오늘은 요런거 먹으니 기쁘네요 이제부터

Seed Sentence: 1	취업해야 할 나이인데 취업하고 싶지가 않아.
Generated Sentence: 해야 할 나이인데 취업하고 싶지가 않아..  이번에도 역시나. 저도 ..이렇게 저번 주말에는 정말 많이 왔어용!오늘

Seed Sentence: 1	우리 회사는 정말 사내 분위기가 좋아.
Generated Sentence: 회사는 정말 사내 분위기가 좋아...그냥 들아.아니 이거야 뭐? 괜히 내가 왜 이렇게 바쁘고 힘들어.들아 괜히 들이 삐져 나오잖

Seed Sentence: 1	오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.
Generated Sentence: 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.네? 넌 뭔가를 하는 거 같기도 하구.그렇다고 뭐라구.그럼 그건 좀 씁씁한 생각이 들어서 그래.근데그

Seed Sentence: 1	회사에서 전공시험을 봤어. 오늘 시험 결과가 나왔어.
Generated Sentence: 전공시험을 봤어. 오늘 시험 결과가 나왔어. 뭘 보고 뭔가 좀 놀라고 싶은데.이번 학기 시험에서 나는 내심 걱정도 했지만 그래도 나는 이 시험은 잘 치러봐야 한다는 생각으로 시험을

Seed Sentence: 1	오늘 입사 면접을 봤어.
Generated Sentence: 입사 면접을 봤어.??응?.어.?그거?어머?응어..응!?어??아뇨! 어.아뇨어..

Seed Sentence: 1	회사에서 나를 참 신뢰하는 것 같아. 그건 기분이 좋아.
Generated 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
main(chunk2, 'augmented_data_chunk2.csv')
files.download('augmented_data_chunk2.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	이번에 상견례를 하게 되었는데 예비 시부모님께서 두 분 다 친절하고 상냥하신 데다가 신세대처럼 재밌는 분들이라 너무 좋았어. 긴장하고 갔는데 너무 다행인 거 있지?
Generated Sentence: 번에 상견례를 하게 되었는데 예비 시부모님께서 두 분 다 친절하고 상냥하신 데다가 신세대처럼 재밌는 분들이라 너무 좋았어. 긴장하고 갔는데 너무 다행인 거 있지?가 정말 잘

Seed Sentence: 1	난 학점이 좋아서 취직이 잘 될 것 같아.
Generated Sentence: 학점이 좋아서 취직이 잘 될 것 같아. 미스터리 아빠랑 함께 놀아줄 녀이렇게 좋은데 왜 안되는지 알 것 같은. 아니

Seed Sentence: 1	이번에 원하는 기업에 취업 성공했어. 취업 준비 기간이 길어질까 봐 걱정했는데 너무 다행이야.
Generated Sentence: 번에 원하는 기업에 취업 성공했어. 취업 준비 기간이 길어질까 봐 걱정했는데 너무 다행이야. 취업스타그램 에서 만나볼수 있는 기회이니만큼 많은 관심 가져줬으니 앞으로도 계속 좋은 취업이 될

Seed Sentence: 1	나한테 여자친구가 생겼어! 선물하고 싶은데 어떤 게 좋을까?
Generated Sentence: 테 여자친구가 생겼어! 선물하고 싶은데 어떤 게 좋을까?.?  이거 진짜 잘 먹었는데요..아직까지 맛있게 잘 먹고있습니당 요즘 핫하다는 치

Seed Sentence: 1	요즘 직장 상사 때문에 걱정이야.
Generated Sentence: 직장 상사 때문에 걱정이야..그래서 오늘 저녁을 먹을까?.그래도 괜찮은데!??!?오늘은 뭐가 더 맛있는가봐!?아 뭔가가 좀 더 맛나겠

Seed Sentence: 1	얼마 전에 친구랑 싸워서 걱정이 많았어.
Generated Sentence: 전에 친구랑 싸워서 걱정이 많았어.그런데 넌 내가 잘해줬지?나는 고개를 흔들었다.그래그럼 잘됐다. 잘했지? 그치. 뭘. 뭘?

Seed Sentence: 1	요즘 직장 다니기 힘들어.
Gene

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
main(chunk3, 'augmented_data_chunk3.csv')
files.download('augmented_data_chunk3.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	남편이 술을 너무 자주 마셔서 걱정인데 본인은 건강하다고 자신하네.
Generated Sentence: 술을 너무 자주 마셔서 걱정인데 본인은 건강하다고 자신하네.그래야 더 맛있게 즐길 수가 있으니이번에 소개팅에 다녀왔습니당

Seed Sentence: 1	남편하고 나하고 열심히 저축해서 남편이 은퇴해도 별걱정 없어.
Generated Sentence: 하고 나하고 열심히 저축해서 남편이 은퇴해도 별걱정 없어.. 괜찮은데. 롯데 현대  한진 등 대기업이 협력사들에게 하도급 대금결제에 대해 원천징수를

Seed Sentence: 1	친구가 나를 많이 도와줘서 좋아
Generated Sentence: 나를 많이 도와줘서 좋아.다이어트레싱 으로 유명한 ... ..의     은

Seed Sentence: 1	나 임신하게 되었어 축하해줄래?
Generated Sentence: 임신하게 되었어 축하해줄래?아직 안아도 되겠지만...아니 진짜...아기랑 같이 놀아야겠다 아기가 엄마한텐 엄마보다 엄마의 힘이 더 세

Seed Sentence: 1	나 있지 결혼을 너무 잘한 것 같아. 남편이 너무 잘해줘.
Generated Sentence: 있지 결혼을 너무 잘한 것 같아. 남편이 너무 잘해줘. .그렇게 해서 괜히 뭔지 알 것 같아서.아직까지 결혼할 생각을 못 했는데 결혼하고 나면 이도

Seed Sentence: 1	결혼을 하고 이렇게 행복할 줄 몰랐어.
Generated Sentence: 하고 이렇게 행복할 줄 몰랐어.아 넌 넌 내일까지 내사랑은 안 하고 내일도 ..내 사랑 내 사랑 넌 나야! 내 사랑은 내가 만들어줘..

Seed Sentence: 1	대학교에서 마음에 드는 친구가 나에게 먼저 말을 걸었어.
Generated Sentence: 에서 마음에 드는 친구가 나에게 먼저 말을 걸었어. 넌 내가 뭘 하는 사람인가?.그런 사람이 어디서 왔지. 뭔데요???!!??

Seed Sentence: 1	우리 회사 선배는 아주 좋은 분 같아.
G

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
main(chunk4, 'augmented_data_chunk4.csv')
files.download('augmented_data_chunk4.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	이번에 운 좋게 수출 계약서에 사인을 받았어.
Generated Sentence: 번에 운 좋게 수출 계약서에 사인을 받았어. 뭔가 특별한 건가?하고 물어보는 순간 그 순간 그건 저기 저기다.아마도 그걸로 만족하고 있었을 것이다.하지만 우리

Seed Sentence: 1	남자 친구와 있으면 마음이 너무 편안해져.
Generated Sentence: 친구와 있으면 마음이 너무 편안해져..그럼 내가 왜 이렇게 섹스를 잘 못하는 거지?내가 어떻게 해야 할지도 모르겠고?난 그 여자랑 같이 섹스가 싫거든.?아니 난 그냥 섹스를

Seed Sentence: 1	외국에 계시는 이모님이 결혼 선물을 보내 주셨어.
Generated Sentence: 계시는 이모님이 결혼 선물을 보내 주셨어.결혼준비하시는 이모님이 뭔데요?아무 것도 모르셨는데요!신부님그램 으로 접속하면

Seed Sentence: 1	나 이번에 회사에서 승진했어. 승진하면서 연봉도 많이 오르게 될 것 같아.
Generated Sentence: 이번에 회사에서 승진했어. 승진하면서 연봉도 많이 오르게 될 것 같아..그래 이직하면 어떻게 할 건가 봐?라며 물었다.그런 거는 하지 않을래. 이직하면 어떻게 하라는 말이

Seed Sentence: 1	드디어 남자 친구랑 결혼해.
Generated Sentence: 남자 친구랑 결혼해.. 괜찮은 거 같아?.그럼 나도 좀 더 행복해 ?아직까지는..그래야겠다 싶어요!이젠.

Seed Sentence: 1	퇴직하면 세계 여행이나 가야겠어.
Generated Sentence: 하면 세계 여행이나 가야겠어. 여행 여행의향 으로여행에스타그램 년 월

Seed Sentence: 1	은퇴하면 책을 하나 쓸 거야.
Generated Sentence: 하면 책을 하나 쓸 거야. 넌 지금 당장 내일 아침 일찍 나가서 책 한권이라도 더 읽고 싶어! 그리고 내일 저녁 일찍 일어나서 책을 읽는 거 그리고 오늘 밤 늦게나 되어야 내일이 되면 다시

Seed Sent

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
main(chunk5, 'augmented_data_chunk5.csv')
files.download('augmented_data_chunk5.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	아직도 주말이면 산 정상에 오를 정도로 체력이 좋아. 내가 체력만큼은 좋다고 자부해.
Generated Sentence: 주말이면 산 정상에 오를 정도로 체력이 좋아. 내가 체력만큼은 좋다고 자부해.그렇게 해서라도 내가 다시 돌아올 거라 믿어. 년 월 일 라디오

Seed Sentence: 1	운동을 안 해도 지금껏 아주 건강히 잘 살고 있어.
Generated Sentence: 안 해도 지금껏 아주 건강히 잘 살고 있어.  요즘은 정말 많은 사람들 저런 사람들한테서 많이 듣던 말이 있죠 저렇게 저럴 때면 항상 이렇게 말하죠 저

Seed Sentence: 1	언제 떠나도 이 생에 미련이 남지 않도록 늘 최선을 다해 살아왔어.
Generated Sentence: 떠나도 이 생에 미련이 남지 않도록 늘 최선을 다해 살아왔어.그래도 괜찮아!?그래 잘했어? 괜히 걱정할 거 아녀도 아니고 괜한

Seed Sentence: 1	모두 내 나이를 알면 깜짝 놀라며 건강의 비법이 무엇이냐고 자주 물어봐.
Generated Sentence: 내 나이를 알면 깜짝 놀라며 건강의 비법이 무엇이냐고 자주 물어봐.그렇지 않아그래? 하고 대답하자 그는 어머어휴!라고 말했다.그러고는 그 자리에서 다시 일어나더니

Seed Sentence: 1	젊었을 때 성질에 비해 지금은 아주 여유로워.
Generated Sentence: 었을 때 성질에 비해 지금은 아주 여유로워..이런 생각을 했거든.그는 다시 한 걸음 뒤로 돌아서서 고개를 들었다.그가 다시 한 발짝 앞으로 다가는 걸 보니 그가 먼저 고개를 숙이고 서 있는 것을 보았다.그는 잠시

Seed Sentence: 1	남편의 당뇨 때문에 운동 겸 등산을 다니면서 좋은 풍경을 보고 와.
Generated Sentence: 당뇨 때문에 운동 겸 등산을 다니면서 좋은 풍경을 보고 와. .이번 주말엔 더워져서 더더더운동복 으로도 좋지만.

Seed Sentence: 1	며느리가 날 간병하게 돼서 모녀처럼 지내려고 노력

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
main(chunk6, 'augmented_data_chunk6.csv')
files.download('augmented_data_chunk6.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	요새 자식들이 나한테 신경을 굉장히 많이 써줘서 기분이 아주 좋아.
Generated Sentence: 자식들이 나한테 신경을 굉장히 많이 써줘서 기분이 아주 좋아.가 이번에 이거 뭐야 이걸 또 사줘야겠다라고 해서 그 이거 사주고 싶어서 아유 내가 너한테 사

Seed Sentence: 1	정말 인생 최고의 날이야. 내 자식들이 여행을 보내줬거든!
Generated Sentence: 인생 최고의 날이야. 내 자식들이 여행을 보내줬거든!. . 이거 내 아들들이랑 나들이할 수 있을까??? 아! 이젠 내 딸들 여행 갈 수 있는 날이지

Seed Sentence: 1	내 자식들이 글쎄 지들끼리 돈을 모아서 좋은 곳으로 날 이사시켜 줬지 뭐야? 정말 너무 뿌듯하다.
Generated Sentence: 자식들이 글쎄 지들끼리 돈을 모아서 좋은 곳으로 날 이사시켜 줬지 뭐야? 정말 너무 뿌듯하다.. 근디 네들이 우리 아들한테 무슨 짓이라도 해. 우리 집에도 좋은 곳이 있어. 아

Seed Sentence: 1	젊어서 고생은 사서도 한다는 말이 맞았어. 괜스레 기분이 좋네.
Generated Sentence: 어서 고생은 사서도 한다는 말이 맞았어. 괜스레 기분이 좋네.? 아까 내가 말한 것 같은 거는. 아 그래.그럼 내가 뭐라 그랬냐면 내가 뭔가를 할 줄 아는 게

Seed Sentence: 1	다행이다! 병원비가 생각보다 적게 나왔어.
Generated Sentence: 행이다! 병원비가 생각보다 적게 나왔어.그녀의 말을 듣고는이미 그놈은 다 죽었고 이제 더는 살아나지 않으리라는 생각을 했다.이제는 더 이상은 살 길이 없는 거야

Seed Sentence: 1	다행이야. 내 병이 노인혜택을 받을 수 있는 질병이래.
Generated Sentence: 행이야. 내 병이 노인혜택을 받을 수 있는 질병이래.. 내 병을 고쳐 줄 사람이 필요하니?아무리 생각한다고 해 봐야 그건 말이야. 내가 이 세상에 태어났다면 나보다 먼저 죽어야 할 사람이

Se

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
main(chunk7, 'augmented_data_chunk7.csv')
files.download('augmented_data_chunk7.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	어버이날에 나에게 카네이션을 달아주는 아이들의 모습에 보람을 느껴.
Generated Sentence: 이날에 나에게 카네이션을 달아주는 아이들의 모습에 보람을 느껴.아무쪼끔 부모님과 가족들의 마음을 헤아려서 좋은 선물도 드리자. 년 월 일일차..

Seed Sentence: 1	아내가 언제나 나를 충분히 배려하고 이해해 줘서 좋아.
Generated Sentence: 언제나 나를 충분히 배려하고 이해해 줘서 좋아. 이번에 나온 샴퓨어는 천연 샴푸는 천연계면활성제와 왁

Seed Sentence: 1	일과 가정 둘 모두에 집중할 수 있는 지금의 삶이 정말 만족스러워.
Generated Sentence: 가정 둘 모두에 집중할 수 있는 지금의 삶이 정말 만족스러워. 아빠랑아침식사 요즘 날씨가 추워지고 있어서다히먹고 뒹구는곳오늘부터 본격적으로 외출하기!!

Seed Sentence: 1	아내와 나는 아이가 없어도 지금 충분히 만족해.
Generated Sentence: 나는 아이가 없어도 지금 충분히 만족해.. 엄마와나아들아침식사 다신샵엄마의아빠일기일상스타 엄마랑아빠일기아빠랑

Seed Sentence: 1	오늘 여자친구를 위해 어렵게 예약한 음식점이 듣던 대로 굉장히 만족스러웠어.
Generated Sentence: 여자친구를 위해 어렵게 예약한 음식점이 듣던 대로 굉장히 만족스러웠어. 예비엄마를 만나서 기뻤던 날이었어? 어제 아침부터  다신오빠랑

Seed Sentence: 1	나는 아내와의 결혼 생활에 매우 만족하고 있어.
Generated Sentence: 아내와의 결혼 생활에 매우 만족하고 있어. 넌 이제부터 내 아내랑 결혼하는 거에 대해서 이야기하고 싶은 게 많아.나는 그 말에 대해 어떻게 대답하겠는가 하고 물었을 때 그녀는 이렇게 말하곤 했던 거야 하고

Seed Sentence: 1	아내와 세운 임신 계획이 아주 잘 맞아떨어져서 너무 만족스러워.
Generated Sentence: 세운 임신 계획이 아주 잘 맞아떨어져서

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
main(chunk8, 'augmented_data_chunk8.csv')
files.download('augmented_data_chunk8.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	일흔의 나이까지 건강하게 살고 있는 나에게 자식들이 여행을 보내줬어.
Generated Sentence: 흔의 나이까지 건강하게 살고 있는 나에게 자식들이 여행을 보내줬어. 여행갈.여행 갈 때는 여행가방을 가지고 가는데 이불 으로도 가방 하나 챙길 수 있어요.그래

Seed Sentence: 1	신체나이가 내 나이보다 훨씬 젊게 나왔어.
Generated Sentence: 나이가 내 나이보다 훨씬 젊게 나왔어..그래도 나처럼 늙고 싶어.이번에는 그 말이 내 귀에 들어왔다.그때 내 가슴은 거리며 떨리고 있는 게 느껴지더니 갑자기 쿵

Seed Sentence: 1	유치원생인 손자의 체육대회에 손자와 함께 이인삼각 달리기를 하기로 했어.
Generated Sentence: 생인 손자의 체육대회에 손자와 함께 이인삼각 달리기를 하기로 했어..그렇게 해서 이삼 일 동안 두 사람은 서로의 손을 잡고 서로가 좋아하는 것을 확인하고 또 즐겼다.그리고 이삼 일은 그 다음날인 지난달 일

Seed Sentence: 1	체육센터에서 운동을 하는데 회원들이 나에게 젊어 보인다고 말했어. 아주 신이 나.
Generated Sentence: 센터에서 운동을 하는데 회원들이 나에게 젊어 보인다고 말했어. 아주 신이 나.가 젊다고 생각하니?는 왜 이렇게 많이 달라요 왜요! 젊어서도 좋아요! 등의 반응이 이어지자

Seed Sentence: 1	딸이 내 건강을 위해 안마기를 사줬어! 너무 신나고 기뻐.
Generated Sentence: 내 건강을 위해 안마기를 사줬어! 너무 신나고 기뻐..  괜찮은 거죠?아뇨 괜한 걱정 말아요이렇게까지 안아준다고 해서 왠만해서는 뭘로 해야 하나

Seed Sentence: 1	친했던 동창 친구들을 만났는데 모두 건강하게 살아있었어. 너무 기뻐.
Generated Sentence: 했던 동창 친구들을 만났는데 모두 건강하게 살아있었어. 너무 기뻐.이런 좋은 추억을 많이 만들었는데.. 그걸 또 다른 친구들 앞에서 이렇게 자랑해

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
main(chunk9, 'augmented_data_chunk9.csv')
files.download('augmented_data_chunk9.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	내 나이에 이렇게 건강할 수 있음에 감사해.
Generated Sentence: 나이에 이렇게 건강할 수 있음에 감사해.이제 내일부터는 내가 할게요.내일이 되면 내일을 위해서이번엔 또 내일 또 내가 해봐요.

Seed Sentence: 1	나 조만간 수술을 해야한대.
Generated Sentence: 조만간 수술을 해야한대. 롯데는 오는 일 오후 시에 쉐프컬쳐 스튜디오를 열고 본격적인 브랜드 론칭제품 쇼룸에 들어간다.지속가능 경영 지속 가능성 지속가능한 성장 등

Seed Sentence: 1	건강검진 결과가 나왔는데 건강하다고 해서 안심이야.
Generated Sentence: 검진 결과가 나왔는데 건강하다고 해서 안심이야.가 건강하다고 해.는 왜 그렇게 안심이죠.?아무것도 모르는 사람..어머나.가 너무 잘 알고 있는거 아니야?

Seed Sentence: 1	아침 프로그램에서 석류즙이 그렇게 좋다는데. 사야겠다.
Generated Sentence: 프로그램에서 석류즙이 그렇게 좋다는데. 사야겠다. . 이거 너무 좋아하세요?아직까지도 이걸 좋아하고 있나보다.어쨌든 오늘이 생일선물이라

Seed Sentence: 1	우리팀이 창업 아이디어 프로젝트에서 일등을 했어 그래서 포상금을 받았는데 기분이 좋아 그동안의 스트레스가 한번에 날아가는 것 같아.
Generated Sentence: 팀이 창업 아이디어 프로젝트에서 일등을 했어 그래서 포상금을 받았는데 기분이 좋아 그동안의 스트레스가 한번에 날아가는 것 같아.가 이제 팀에 들어와서 이제 그게 뭐지 뭐라고 했나.근깐 팀

Seed Sentence: 1	이번 프로젝트 성공적이었다고 사장님께 격려금 받았어. 너무 기분 좋아.
Generated Sentence: 프로젝트 성공적이었다고 사장님께 격려금 받았어. 너무 기분 좋아.프로젝스 .인생템프로덕션

Seed Sentence: 1	드디어 승진 대상자에 선정됐어! 너무 신나!
Generated Sentence: 승진 대상자에 선정됐어! 너

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
main(chunk10, 'augmented_data_chunk10.csv')
files.download('augmented_data_chunk10.csv')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Seed Sentence: 1	그녀가 나랑 결혼하겠대. 너무 신나.
Generated Sentence: 나랑 결혼하겠대. 너무 신나. 아줌마라 아들바보 딸랑구두 요즘 신발 으로신세계몰 에서 구매한 신상품들

Seed Sentence: 1	업무가 너무 과중하긴 한데 그래도 시작해봐야지.
Generated Sentence: 너무 과중하긴 한데 그래도 시작해봐야지..먹그램그램  ..  ..토요일 .

Seed Sentence: 1	회사에서 명절 보너스로 백 퍼센트 상여금이 나왔어. 생각지도 못한 보상이라 너무 기분이 좋아.
Generated Sentence: 명절 보너스로 백 퍼센트 상여금이 나왔어. 생각지도 못한 보상이라 너무 기분이 좋아.. 오늘은 정말 뭘 먹었어요?이건 뭔가

Seed Sentence: 1	오늘 가고 싶은 회사 면접을 봤어.
Generated Sentence: 가고 싶은 회사 면접을 봤어..그럼.응응응?어떻든 응?그거 땜시 면접 볼까요!! 오늘 아침신문 살펴봤어요.오늘 신문에서 보도했던 내용 중

Seed Sentence: 1	아빠가 오늘 엄마랑 엄청나게 싸웠는데 싸우는 도중에 이혼 얘기까지 나왔었어.
Generated Sentence: 빠가 오늘 엄마랑 엄청나게 싸웠는데 싸우는 도중에 이혼 얘기까지 나왔었어.. 아빤 아빠랑 아빠한테 이혼 얘기를 할려고 그러니까. 넌 뭐냐고 물어봤지 뭐냐? 아빠한테

Seed Sentence: 1	오늘도 예전과 같이 교실에서 학교폭력이 일어날 뻔했어. 그런데 선생님이 그 현장을 목격해서 너무 다행이야.
Generated Sentence: 도 예전과 같이 교실에서 학교폭력이 일어날 뻔했어. 그런데 선생님이 그 현장을 목격해서 너무 다행이야.는 괜히 뭘 보고 있나.그냥 를 그냥 놔두려고 그러

Seed Sentence: 1	원하던 직장에 합격해서 좋다.
Generated Sentence: 하던 직장에 합격해서 좋다.. !! 직장에 다니면 직장이동력기 가 필요한데..이런거 안쓰는 사람이 많다!이제 직장에

Seed 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>